In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
from elasticsearch import Elasticsearch
from elasticsearch import helpers
indexingTables = Elasticsearch()

In [2]:
def getAccuracy(idRankedTables, idQueryGoal):
    
    accuracy = 0

    for idTable in idRankedTables:
    
        if idTable == idQueryGoal:
    
            accuracy = 1
            break;

    return accuracy

In [11]:
indexingTables.indices.close(index='tables')
indexingTables.indices.put_settings(index='tables', body={"index": {"similarity": {"default": {"type": "classic"}}}})
indexingTables.indices.open(index='tables')

{'acknowledged': True, 'shards_acknowledged': True}

In [4]:
def searchIndexingMLT(query, tableField, topK):
        
    result = indexingTables.search(
    index="tables", 
    body = {
        "_source": ["tablePgID"],
        "from" : 0,
        "size" : topK,
        "query": {
            "more_like_this" : {
                "fields" : [tableField],
                "like" : "["+query+"]",
                "min_term_freq" : 0,
                "min_doc_freq":0,
                "max_query_terms":50
            }
        }
    }
    )
    
    return result

In [5]:
def searchIndexing(query, tableField, topK):
    
    result= indexingTables.search(
        index="tables", 
        body = {
        "_source": ["tablePgID"],
        "from" : 0,
        "size" : topK,
        "query": {
            "multi_match":{
              "type": "most_fields",
              "query":    query, 
              "fields": ["tableCaption","tableSectionTitle","tableHeader"] 
              #"fields": [tableField]
            }
        }
    })
    
    return result

In [6]:
articles = pd.read_csv('articlesDataset/articlesTestDataset', delimiter=',', header=None)
formattedArticles = articles.iloc[:,:].values

In [8]:
def executeSearch(tableField, topK):

    accuracy = []

    for articles in (formattedArticles):
    
        articleKey = articles[0]
        
        articleTitle = str(articles[1])
    
        articleText = str(articles[2])
        
        result = searchIndexing(articleText,tableField,topK)
    
        idRankedTables = []
    
        for hit in result['hits']['hits']:
    
            tablePgId = hit['_source']['tablePgID']
    
            idRankedTables.append(tablePgId)
    
        accuracy.append(getAccuracy(idRankedTables,articleKey))

    print(str(round(np.mean(accuracy),4))+" (±) "+str(round(np.std(accuracy),4)))

In [9]:
tableField = ["tablePgTitle","tableSectionTitle","tableCaption","tableHeader","tableBody"]
topK = [1,10,100,1000]

In [13]:
for k in topK:
    
    print("Acc@"+str(k))
    
    for field in tableField:
        
        executeSearch(field,k)

Acc@1


RequestError: RequestError(400, 'search_phase_execution_exception', 'failed to create query: {\n  "multi_match" : {\n    "query" : "close skip main content sign subscribe search jobs guardian jobs change edition switch edition switch edition switch int beta edition edition switch edition switch australia edition switch international beta guardian home soccer live scores tables competitions results fixtures clubs home world opinion sports selected soccer selected tech arts lifestyle fashion business travel environment science browse sections close zimbabwe peter ndlovu bruce grobbelaar made zimbabwe dare dream early country finally golden generation players stood verge quaifying africa cup nations world cup bruce grobbelaar left shares joke zimbabwe team mate peter ndlovu photograph howard burditt reuters ian hawkey wednesday august edt share facebook share twitter share via email share pinterest share linkedin share google share whatsapp following extract ian hawkey article issue nine 